### Test Model Using vec2vec lstm
- seq2seq lstm
- cnn + lstm



In [ ]:
import numpy as np
import pandas as pd
import time
from tensorflow.examples.tutorials.mnist import input_data

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop

import math
import warnings
import matplotlib.pyplot as plt
import itertools
from datetime import datetime, timedelta
from pandas.api.types import is_timedelta64_dtype
warnings.filterwarnings("ignore")
print("done")

In [ ]:
test = pd.read_csv('./data_pos.csv')
df_copy = test.copy()
df_copy.transacted_date = pd.to_datetime(df_copy.transacted_date)
print("done")

df = df_copy
test_groupby_date_store = df.groupby(['transacted_date', 'store_id'])['amount'].sum()
test_groupby_date_store = test_groupby_date_store.reset_index()
test_groupby_date_store = test_groupby_date_store.set_index('transacted_date')
store_list = test_groupby_date_store.store_id.unique()
store_list.sort()
store_list

In [ ]:
test_start = datetime(2018,12,1)
test_end = datetime(2019,2,28)

# print(test_groupby_date_store.head())
ts_train = test_groupby_date_store[test_groupby_date_store.index<test_start]
ts_test = test_groupby_date_store[test_groupby_date_store.index>=test_start]
ts_test = ts_test[ts_test.index<test_end]

# train test 개수가 적합한지 확인
threshold = (test_start - datetime(2016,6,1)).days
ts_train_number = ts_train.groupby(['store_id']).count()
ts_train_number = ts_train_number[ts_train_number.amount>0.5*threshold]
store_list_train = ts_train_number.index.unique()
threshold2 = (test_end - test_start).days
ts_test_number = ts_test.groupby(['store_id']).count()
ts_test_number = ts_test_number[ts_test_number.amount>0.5*threshold2]
store_list_test = ts_test_number.index.unique()
store_list = [store_id for store_id in store_list_test if store_id in store_list_train]
print("store list size",len(store_list))

In [ ]:
print(threshold2)
true_sum = {}
for store_id in store_list[:]:
    
    sales = ts_test[ts_test.store_id==store_id].amount
    true_sum[store_id]=sales.sum()*(threshold2/sales.count())
test_sum = {}
print("done")

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import MinMaxScaler

def split_data(data, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(data)-n_steps_in-n_steps_out):
        a = dataset[i:(i+n_steps_in)]
        X.append(a)
        y.append(dataset[i+n_steps_in:i+n_steps_in+n_steps_out].sum())
    return np.array(X), np.array(y)

# def split_data(data):
#     X, y = [], []
#     for i in range(len(data)-7-7):
#         a = data[i:(i+7)]
#         b = data[i:(i+14)].sum()
#         c = data[i:(i+28)].sum()
#         a = np.std(data[i:(i+7)])
#         b = np.std(data[i:(i+14)])
#         c = np.std(data[i:(i+28)])
#         X.append([a])
#         y.append(dataset[i+28:i+28+92].sum())
#         y.append(dataset[i+7:i+7+7])

#     return np.array(X), np.array(y)


for store_id in store_list[:1]:
    prediction_i = None    
    test_df = ts_train[ts_train.store_id==store_id]
    dataset = test_df.values
    dataset = dataset.astype('float64')
    scaler = MinMaxScaler(feature_range=(0,1))
    dataset = scaler.fit_transform(dataset)
    dataset = dataset[:,1]
    
    n_steps_in = 7
    n_steps_out = 7
    X,y = split_data(dataset,n_steps_in,n_steps_out)
    n_features = 1
    X = X.reshape((X.shape[0],n_steps_in,n_features))
    
#     y = y.reshape((y.shape[0], y.shape[1], 1))
#     model = Sequential()
#     model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps_in,n_features)))
#     model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#     model.add(MaxPooling1D(pool_size=2))
#     model.add(Flatten())
#     model.add(RepeatVector(n_steps_out))

    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(n_steps_in, n_features)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_features))
    model.compile(loss='mse', optimizer='adam')
    model.fit(X,y, epochs=50, batch_size=1, verbose=2)
    
print("done")

In [ ]:
pre_sum = []
true_sum = []

for i in range(1):
    x_input = X[i]
    x_input = x_input.reshape((1,n_steps_in,n_features))
    model.predict(x_input, verbose=2)
    true_sum.append([0,y[i].sum()])
    pre_sum.append([0,yhat.sum()])


pre_sum = scaler.inverse_transform(pre_sum)
true_sum = scaler.inverse_transform(true_sum)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(pre_sum[:,1],true_sum[:,1])
print("mae",mae)

plt.figure(figsize=(22,10))
plt.plot(true_sum[:],label = "true_sum")
plt.plot(pre_sum[:],label = "predict_sum")
plt.title("lstm")
plt.xlabel("case")
plt.ylabel("3month amount")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def monthly_data(data):
    X = []
    for i in range(0,len(data)-28*3,28*3):
        X.append(data[i:i+28*3].sum())
    return np.array(X)

prediction_i = None    
test_df = ts_train[ts_train.store_id==1104]
# print(test_df)
dataset = test_df.values
dataset = dataset.astype('float64')
scaler = MinMaxScaler(feature_range=(0,1))
dataset = scaler.fit_transform(dataset)
dataset = dataset[:,1]

X = monthly_data(dataset)
X
plt.figure(figsize=(22,10))
plt.plot(X,label = "monthly")
plt.title("lstm")
plt.xlabel("case")
plt.ylabel("3month amount")
plt.legend()
plt.tight_layout()
plt.show()